In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import xgboost as xgb

In [ ]:
df = pd.read_csv('data/run1.csv')
df.head(2)

,gate_type,voltage,thickness,clock_cycle,t_rise,t_fall,t_delay
0,buffer,0.3,0.1,100,3.131324e-12,3.17279e-12,9.921646e-12
1,buffer,0.4,0.1,100,8.473565e-13,1.456597e-12,7.269524e-12


In [ ]:
#these are the bad t_fall values in nand4 that need to be dropped
nand4_drop_idxs = ((df['gate_type'] == 'nand4') & (df['t_fall'] == ' FAILED')).values
nand4_drop_idxs.sum()

36

In [ ]:
df = df.loc[~nand4_drop_idxs].reset_index(drop=True)
df.shape

(11304, 7)

In [ ]:
def get_gate(gate_type='buffer'): return df[df.gate_type == gate_type]

In [ ]:
unique_gates = list(df.gate_type.unique())
unique_gates

['buffer',
 'inv_x1',
 'inv_x2',
 'inv_x4',
 'nand2',
 'nand3',
 'nand4',
 'nor2',
 'nor3',
 'nor4']

In [ ]:
def get_gate_xy(gate_type='buffer'):
    y_drop = 't_fall' if gate_type=='buffer' else 't_rise'
    df = get_gate(gate_type)
    x_cols = ['clock_cycle', 'thickness', 'voltage']
    y_cols = ['t_delay', 't_rise', 't_fall']
    y_cols.remove(y_drop)
    x = np.vstack(df[x_cols].values).astype('float')
    y = np.vstack(np.vstack(df[y_cols].values)).astype('float')
    return x, y

In [ ]:
x, y = get_gate_xy('nor3')
x.shape, y.shape

((1134, 3), (1134, 2))

In [ ]:
gate_dict = dict.fromkeys(unique_gates)
gate_dict

{'buffer': None,
 'inv_x1': None,
 'inv_x2': None,
 'inv_x4': None,
 'nand2': None,
 'nand3': None,
 'nand4': None,
 'nor2': None,
 'nor3': None,
 'nor4': None}

In [ ]:
for gate in unique_gates: 
    gate_dict[gate] = get_gate_xy(gate_type=gate)

In [ ]:
def ridge_reg(x_train, y_train, x_valid, y_valid):
    print('Doing Ridge Regression.....')
    ridge = Ridge(alpha = 1)
    ridge.fit(x_train, y_train)
    print('training score = ', ridge.score(x_train, y_train))
    y_pred_ridge = ridge.predict(x_valid)
    mse_ridge = mean_squared_error(y_valid, y_pred_ridge) 
    print('test score =', ridge.score(x_valid, y_valid)) 

def do_rf(x_train, y_train, x_valid, y_valid):
    print('\n\ndoing RF.....')
    y_train = y_train / y_train.min()
    y_valid = y_valid / y_valid.min()
    RF = RandomForestRegressor(max_depth=100, random_state=0)
    RF.fit(x_train, y_train)
    print('trainig score = ', RF.score(x_train, y_train))
    y_pred = RF.predict(x_valid)
    mse = mean_squared_error(y_valid, y_pred)
    r2 = r2_score(y_valid, y_pred) # coefficient of determination
    print('test score =', RF.score(x_valid, y_valid))
    print(f'the mse is {mse}')
    print(f"the r2 score is: {r2}")
    return y_pred


def do_xgb(x_train, y_train, x_valid, y_valid):
    print('\n\nboosting the trees.....')
    y_train = y_train / y_train.min()
    y_valid = y_valid / y_valid.min()
    regressor=xgb.XGBRegressor(eval_metric='rmse')
    regressor.fit(x_train, y_train)
    train_score = regressor.score(x_train, y_train) ** 2 #bc using RMSE
    y_pred = regressor.predict(x_valid)
    mse = mean_squared_error(y_valid, y_pred)
    r2 = r2_score(y_valid, y_pred) # coefficient of determination
    valid_score = regressor.score(x_valid, y_valid)

    print(f'the training score is {train_score:3f}')    
    print(f"the validation score is {valid_score:3f}")
    print(f'the mse is {mse:3f}')
    print(f"the r2 score is: {r2:3f}")
    return y_pred



def train_models(gate_type='buffer', random_state=0):
    print(f'-----Analyzing {gate_type} gates-----\n')
    x, y = gate_dict[gate_type]
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, 
                                                        random_state=random_state, 
                                                        test_size=0.1)    
    ridge_reg(x_train, y_train, x_valid, y_valid)
    y_pred = do_rf(x_train, y_train, x_valid, y_valid)
    _ = do_xgb(x_train, y_train, x_valid, y_valid)
    print('\n\n')
    #return y_pred

In [ ]:
x, y = gate_dict['buffer']
x.shape, y.shape

((1134, 3), (1134, 2))

In [ ]:
train_models('buffer', random_state=123)

-----Analyzing buffer gates-----

Doing Ridge Regression.....
training score =  0.6868883807464006
test score = 0.7054164133458096


doing RF.....
trainig score =  1.0
test score = 1.0
the mse is 1.5906988750755812e-27
the r2 score is: 1.0


boosting the trees.....
the training score is 0.999999
the validation score is 1.000000
the mse is 0.000005
the r2 score is: 1.000000





In [ ]:
[train_models(gate_type, random_state=123) for gate_type in gate_dict.keys() ]

-----Analyzing buffer gates-----

Doing Ridge Regression.....
training score =  0.6868883807464006
test score = 0.7054164133458096


doing RF.....
trainig score =  1.0
test score = 1.0
the mse is 1.5906988750755812e-27
the r2 score is: 1.0


boosting the trees.....
the training score is 0.999999
the validation score is 1.000000
the mse is 0.000005
the r2 score is: 1.000000



-----Analyzing inv_x1 gates-----

Doing Ridge Regression.....
training score =  0.7802399691292567
test score = 0.7843436443699999


doing RF.....
trainig score =  1.0
test score = 1.0
the mse is 1.565594803982376e-28
the r2 score is: 1.0


boosting the trees.....
the training score is 1.000000
the validation score is 1.000000
the mse is 0.000005
the r2 score is: 1.000000



-----Analyzing inv_x2 gates-----

Doing Ridge Regression.....
training score =  0.7799249311934082
test score = 0.7837168811482245


doing RF.....
trainig score =  1.0
test score = 1.0
the mse is 1.7518788573822228e-28
the r2 score is: 1.0


b

[None, None, None, None, None, None, None, None, None, None]